In [ ]:
# Stopword function


def stopword(string):
    from nltk.corpus import stopwords
    res = [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(res)

In [ ]:
# Download and import tokenizer libraries
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
import nltk
# nltk.download('wordnet')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

In [ ]:
# Initialize lemmatizer
from nltk.stem import WordNetLemmatizer

wl = WordNetLemmatizer()

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [ ]:
# Tokenize Summaries
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    res = [wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(res)

In [ ]:
# Text pre-processing
def get_pre_preprocessing(df1,col):
    import pandas as pd
    import re
    from nltk.tokenize import RegexpTokenizer
    import numpy as np
    df = df1.copy()
# Convert Summary data to type(str)
    df[col] = df[col].apply(lambda x: str(x))
# Remove html tags and leading 'b's
    df[col] = df[col].apply(lambda row: re.compile(r'<.*?>').sub(r' ',row))
# Remove special characters, replace with space
    df[col] = df[col].apply(lambda row: re.compile(r'\W').sub(r' ',row))
# Remove single characters
    df[col] = df[col].apply(lambda row: re.compile(r'\s+[a-zA-Z]\s+').sub(r' ',row))
# Remove stil characters from start
    df[col] = df[col].apply(lambda row: re.compile(r'\^[a-zA-Z]\s+').sub(r' ',row))
# Clean up multi-space gaps
    df[col] = df[col].apply(lambda row: re.compile(r'\s+').sub(r' ',row))
# Converting to lower case
    df[col] = df[col].apply(lambda row: row.lower())
# Apply stopwords func to Summaries
    df[col] = df[col].apply(lambda text: stopword(text))
# Apply lemmatizer func to Summaries
    df[col] = df[col].apply(lambda text: lemmatizer(text))
# Initialize tokenizer and tokenize summary data
    tokenizer = RegexpTokenizer(r'\w+')
#apply tokinizer to text data
    df[col] = df[col].apply(tokenizer.tokenize)
# Create token length count column
    df['token_count'] = [len(x) for x in df[col]]
# Calculate standard deviation of token counts
    print(pd.DataFrame.std(df.token_count, ddof=2))
# Checking standard deviation using np
    arr = pd.Series.to_numpy(df.token_count)
    print(np.std(arr, ddof=2))
#add the tokenize column to the original column 
    df1['token'] = df[col]
    return df1